In [1]:
import pickle
import time
import torch
import numpy as np
from argparse import ArgumentParser
from double_net.double_net import DoubleNet, train_loop, test_loop_random_start, test_loop
from double_net import datasets as ds
import double_net.plot_utils as pu
import matplotlib.pyplot as plt

In [2]:
train = True
test = True

In [3]:
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [5]:
device

'cuda'

In [6]:
parser = ArgumentParser()
parser.add_argument('--random-seed', type=int, default=0)
parser.add_argument('--num-examples', type=int, default=131072 * 4)
parser.add_argument('--test-num-examples', type=int, default=10000)
parser.add_argument('--test-iter', type=int, default=5)
parser.add_argument('--n-agents', type=int, default=2)
parser.add_argument('--n-items', type=int, default=3)
parser.add_argument('--num-epochs', type=int, default=100)
parser.add_argument('--batch-size', type=int, default=128 * 32)
parser.add_argument('--test-batch-size', type=int, default=10000)
parser.add_argument('--model-lr', type=float, default=1e-3)
parser.add_argument('--misreport-lr', type=float, default=1e-1)
parser.add_argument('--misreport-iter', type=int, default=25)
parser.add_argument('--test-misreport-iter', type=int, default=2000)
parser.add_argument('--rho', type=float, default=1.0)
parser.add_argument('--rho-incr-iter', type=int, default=2)
parser.add_argument('--rho-incr-amount', type=float, default=1.0)
parser.add_argument('--lagr-update-iter', type=int, default=100)
parser.add_argument('--rgt-start', type=int, default=0)
parser.add_argument('--sinkhorn-epsilon', type=float, default=5e-2)
# parser.add_argument('--sinkhorn-rounds', type=int, default=20)
parser.add_argument('--sinkhorn-tol', type=float, default=0.01)

dataset_name = ""
args = parser.parse_args(args=[])
args

Namespace(batch_size=4096, lagr_update_iter=100, misreport_iter=25, misreport_lr=0.1, model_lr=0.001, n_agents=2, n_items=3, num_epochs=100, num_examples=524288, random_seed=0, rgt_start=0, rho=1.0, rho_incr_amount=1.0, rho_incr_iter=2, sinkhorn_epsilon=0.05, sinkhorn_tol=0.01, test_batch_size=10000, test_iter=5, test_misreport_iter=2000, test_num_examples=10000)

In [7]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

item_ranges = ds.preset_valuation_range(args.n_agents, args.n_items, dataset_name)
clamp_op = ds.get_clamp_op(item_ranges)

model = DoubleNet(
    args.n_agents, args.n_items, item_ranges, args.sinkhorn_epsilon, args.sinkhorn_tol, marginal_choice='exact_one'
).to(device)

train_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.num_examples, item_ranges).to(device)
train_loader = ds.Dataloader(train_data, batch_size=args.batch_size, shuffle=True)

In [8]:
import os
dir_name = 'scaled_exact_one_2x3_experiment_data/'
os.makedirs(dir_name, exist_ok=True)

In [9]:

if train:
    start_time = time.time()
    mean_rgt, mean_pay, lagr_mults = train_loop(model, train_loader, args, device=device)
    end_time = time.time()
    with open(dir_name+'time.txt', 'w') as f:
        print(f"runtime: {end_time - start_time}", file=f)

100%|██████████| 100/100 [2:37:54<00:00, 94.75s/it] 


{'regret_max': 0.5398610830307007, 'regret_mean': 0.04150930047035217, 'regret_mults': tensor([[6.5384, 6.5607]], device='cuda:0'), 'payment': 0.30793923139572144}
{'regret_max': 0.13323211669921875, 'regret_mean': 0.020696990191936493, 'regret_mults': tensor([[8.8280, 8.5167]], device='cuda:0'), 'payment': 0.24025686085224152}
{'regret_max': 0.08781695365905762, 'regret_mean': 0.009107845835387707, 'regret_mults': tensor([[10.5312,  9.9670]], device='cuda:0'), 'payment': 0.23520201444625854}
{'regret_max': 0.07362759113311768, 'regret_mean': 0.006394682452082634, 'regret_mults': tensor([[13.6271, 12.9795]], device='cuda:0'), 'payment': 0.23903131484985352}
{'regret_max': 0.06508219242095947, 'regret_mean': 0.0049968985840678215, 'regret_mults': tensor([[15.0256, 14.2970]], device='cuda:0'), 'payment': 0.24433918297290802}
{'regret_max': 0.06649917364120483, 'regret_mean': 0.004285966511815786, 'regret_mults': tensor([[16.5534, 16.0264]], device='cuda:0'), 'payment': 0.2472829222679138

In [10]:
if not train:
    model = DoubleNet.load(dir_name).cuda()
else:
    model.save(dir_name)
    to_pkl_lst = [mean_rgt, mean_pay, lagr_mults]
    for i, fname in enumerate(['mean_rgt', 'mean_pay', 'lagr_mults']):
        with open(dir_name + fname, 'wb') as fp:
            pickle.dump(to_pkl_lst[i], fp)

In [11]:
%%time
# model.sinkhorn_rounds = 100
# model.sinkhorn_epsilon = 1e-2
if test:
    test_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    # test_data = torch.load(dir_name + 'test_data').to(device=device)
    cpu_test_data = test_data.clone().to(device='cpu')

    test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop(model, test_loader, args, device=device)
    print(result)

1it [00:50, 50.15s/it]


{'payment_mean': 0.19130839407444, 'regret_mean': 0.004152871668338776, 'regret_max': 0.052350521087646484}
CPU times: user 50.2 s, sys: 16 ms, total: 50.2 s
Wall time: 50.2 s


In [12]:
# %%time
if test:
    args.test_num_examples = 1000
    args.test_batch_size = 1000
    args.test_misreport_iter = 1000
    args.misreport_lr = 1e-1
    test_data2 = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    random_starts = [test_data2]
    for i in range(10):
        random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
    test_loader = ds.Dataloader(test_data2, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
    print(result)

1it [04:34, 274.39s/it]


{'payment_mean': 0.18918870389461517, 'regret_mean': 0.004495672881603241, 'regret_max': 0.047275155782699585}


In [13]:
converged = model.check_convergence(torch.rand(10000,2,3).cuda())

In [14]:
converged[0].max()

tensor(0.0051, device='cuda:0')

In [15]:
converged[1].max()

tensor(8.3446e-07, device='cuda:0')

In [16]:
model.cuda()
model.sinkhorn_epsilon=5e-3
if test:
    args.test_num_examples = 1000
    args.test_batch_size = 1000
    args.test_misreport_iter = 1000
    args.misreport_lr = 1e-1
    test_data2 = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    random_starts = [test_data2]
    for i in range(10):
        random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
    test_loader = ds.Dataloader(test_data2, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
    print(result)

1it [12:34, 754.04s/it]


{'payment_mean': 0.21054710447788239, 'regret_mean': 0.13626669347286224, 'regret_max': 0.3283197283744812}
